<a href="https://colab.research.google.com/github/viibhutisharma30/Virtual_Wardrobe_Stylist_Model/blob/main/Main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import joblib

# Load the dataset
data = pd.read_csv('wardrobe_training_large.csv')

# Initialize LabelEncoders
le_item_type = LabelEncoder()
le_color = LabelEncoder()
le_style = LabelEncoder()
le_occasion = LabelEncoder()
le_season = LabelEncoder()
le_body_type = LabelEncoder()
le_combination = LabelEncoder()

# Encode categorical features
data['item_type'] = le_item_type.fit_transform(data['item_type'])
data['color'] = le_color.fit_transform(data['color'])
data['style'] = le_style.fit_transform(data['style'])
data['occasion'] = le_occasion.fit_transform(data['occasion'])
data['season'] = le_season.fit_transform(data['season'])
data['body_type'] = le_body_type.fit_transform(data['body_type'])
data['recommended_combination'] = le_combination.fit_transform(data['recommended_combination'])

# Split the dataset into two parts: one for tops and one for bottoms
tops = data[data['item_type'].isin(le_item_type.transform(['shirt', 't-shirt', 'blouse']))]
bottoms = data[data['item_type'].isin(le_item_type.transform(['jeans', 'pants', 'shorts', 'skirt']))]

# Train the Random Forest model for tops (predicting bottoms)
X_tops = tops[['item_type', 'color', 'style', 'occasion', 'season', 'body_type']]
y_tops = tops['recommended_combination']

X_train_tops, X_test_tops, y_train_tops, y_test_tops = train_test_split(X_tops, y_tops, test_size=0.2, random_state=42)
top_model = RandomForestClassifier(
    random_state=42,
    n_estimators=50,          # Reduced number of trees
    max_depth=5,              # Limit the depth of trees
    min_samples_split=10,     # Increase minimum samples required to split a node
    min_samples_leaf=5        # Increase minimum samples in each leaf node
)
top_model.fit(X_train_tops, y_train_tops)

train_accuracy_tops = top_model.score(X_train_tops, y_train_tops) * 100
test_accuracy_tops = top_model.score(X_test_tops, y_test_tops) * 100

print(f"Top model training accuracy: {train_accuracy_tops:.2f}%")
print(f"Top model test accuracy: {test_accuracy_tops:.2f}%")

# Save the top model and encoders
joblib.dump(top_model, 'top_model.pkl')

# Train the Random Forest model for bottoms (predicting tops)
X_bottoms = bottoms[['item_type', 'color', 'style', 'occasion', 'season', 'body_type']]
y_bottoms = bottoms['recommended_combination']

X_train_bottoms, X_test_bottoms, y_train_bottoms, y_test_bottoms = train_test_split(X_bottoms, y_bottoms, test_size=0.2, random_state=42)
bottom_model = RandomForestClassifier(
    random_state=42,
    n_estimators=50,          # Reduced number of trees
    max_depth=5,              # Limit the depth of trees
    min_samples_split=10,     # Increase minimum samples required to split a node
    min_samples_leaf=5        # Increase minimum samples in each leaf node
)
bottom_model.fit(X_train_bottoms, y_train_bottoms)

train_accuracy_bottoms = bottom_model.score(X_train_bottoms, y_train_bottoms) * 100
test_accuracy_bottoms = bottom_model.score(X_test_bottoms, y_test_bottoms) * 100

print(f"Bottom model training accuracy: {train_accuracy_bottoms:.2f}%")
print(f"Bottom model test accuracy: {test_accuracy_bottoms:.2f}%")

# Save the bottom model and encoders
joblib.dump(bottom_model, 'bottom_model.pkl')

# Save LabelEncoders
joblib.dump(le_item_type, 'le_item_type.pkl')
joblib.dump(le_color, 'le_color.pkl')
joblib.dump(le_style, 'le_style.pkl')
joblib.dump(le_occasion, 'le_occasion.pkl')
joblib.dump(le_season, 'le_season.pkl')
joblib.dump(le_body_type, 'le_body_type.pkl')
joblib.dump(le_combination, 'le_combination.pkl')


Top model training accuracy: 93.55%
Top model test accuracy: 75.00%
Bottom model training accuracy: 89.47%
Bottom model test accuracy: 70.00%


['le_combination.pkl']

In [ ]:
def predict_outfit(item_type, color, style, occasion, season, body_type):
    # Load the saved encoders
    le_item_type = joblib.load('le_item_type.pkl')
    le_color = joblib.load('le_color.pkl')
    le_style = joblib.load('le_style.pkl')
    le_occasion = joblib.load('le_occasion.pkl')
    le_season = joblib.load('le_season.pkl')
    le_body_type = joblib.load('le_body_type.pkl')
    le_combination = joblib.load('le_combination.pkl')

    # Encode the input features using the saved LabelEncoders
    item_type_enc = le_item_type.transform([item_type])[0]
    color_enc = le_color.transform([color])[0]
    style_enc = le_style.transform([style])[0]
    occasion_enc = le_occasion.transform([occasion])[0]
    season_enc = le_season.transform([season])[0]
    body_type_enc = le_body_type.transform([body_type])[0]

    input_features = [[item_type_enc, color_enc, style_enc, occasion_enc, season_enc, body_type_enc]]

    # Check if the input is a top or a bottom
    if item_type in ['shirt', 't-shirt', 'blouse']:
        # If input is a top, load the bottom model to predict a bottom
        top_model = joblib.load('top_model.pkl')
        prediction = top_model.predict(input_features)
    else:
        # If input is a bottom, load the top model to predict a top
        bottom_model = joblib.load('bottom_model.pkl')
        prediction = bottom_model.predict(input_features)

    recommended_combination = le_combination.inverse_transform(prediction)[0]
    return recommended_combination


In [ ]:
import cv2
import mediapipe as mp
import numpy as np
import joblib

# Initialize mediapipe pose
mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils

def classify_body_type(landmarks):
    if landmarks:
        shoulder_width = np.linalg.norm(np.array([landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,
                                                  landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]) -
                                        np.array([landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x,
                                                  landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y]))
        # Example logic: Classify based on shoulder width (you can modify the logic as needed)
        if shoulder_width > 0.15:
            return "athletic"
        else:
            return "average"
    return "average"

# Initialize the webcam for pose detection
cap = cv2.VideoCapture(0)

# Start the pose detection loop
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False

        results = pose.process(image)

        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        # Draw pose landmarks on the image
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                  mp_drawing.DrawingSpec(color=(245, 117, 66), thickness=2, circle_radius=2),
                                  mp_drawing.DrawingSpec(color=(245, 66, 230), thickness=2, circle_radius=2))

        # If landmarks are detected, classify the body type
        if results.pose_landmarks:
            body_type = classify_body_type(results.pose_landmarks.landmark)
            cv2.putText(image, f'Body Type: {body_type}', (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2, cv2.LINE_AA)

            # Display the video feed
            cv2.imshow('Mediapipe feed', image)

            # Wait for user input to trigger the prediction (e.g., pressing 'p' key)
            if cv2.waitKey(10) & 0xFF == ord('p'):
                # Example inputs for prediction
                item_type = 'shorts'  # Example input for item type
                color = 'white'  # Example input for color
                style = 'casual'  # Example input for style
                occasion = 'daily'  # Example input for occasion
                season = 'summer'  # Example input for season

                # Predict the outfit using the detected body type and other features
                predicted_combination = predict_outfit(item_type, color, style, occasion, season, body_type)
                print(f"Predicted combination based on detected body type: {predicted_combination}")

        # If the user presses the 'q' key, exit the loop
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()


C:\Users\ASUS\Desktop\jupyter_notebooks\env\Lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '
C:\Users\ASUS\Desktop\jupyter_notebooks\env\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
C:\Users\ASUS\Desktop\jupyter_notebooks\env\Lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


Predicted combination based on detected body type: t-shirt
